In [1]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [2]:
%store -r model_name

In [3]:
import boto3
client = boto3.client('sagemaker')

In [4]:
response = client.describe_model(ModelName=model_name)
response

{'ModelName': 'model-ACBOE',
 'PrimaryContainer': {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-05-24-14-02-24-886/output/model.tar.gz',
  'Environment': {}},
 'ExecutionRoleArn': 'arn:aws:iam::581320662326:role/SuperAdminRole',
 'CreationTime': datetime.datetime(2021, 5, 24, 14, 16, 28, 85000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/model-acboe',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': '1ff5ee42-077b-482f-8917-1f01cb75aa3a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1ff5ee42-077b-482f-8917-1f01cb75aa3a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '484',
   'date': 'Mon, 24 May 2021 14:50:29 GMT'},
  'RetryAttempts': 0}}

In [5]:
container = response['PrimaryContainer']['Image']
container

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1'

In [6]:
model_data = response['PrimaryContainer']['ModelDataUrl']
model_data

's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-05-24-14-02-24-886/output/model.tar.gz'

In [7]:
model = sagemaker.model.Model(
    name=model_name,
    image_uri=container,
    model_data=model_data,
    role=role,
    sagemaker_session=session
)

{'model_data': 's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-05-24-14-02-24-886/output/model.tar.gz',
 'image_uri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1',
 'role': 'arn:aws:iam::581320662326:role/SuperAdminRole',
 'predictor_cls': None,
 'env': {},
 'name': 'model-ACBOE',
 '_base_name': None,
 'vpc_config': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7fa440769ed0>,
 'endpoint_name': None,
 '_is_compiled_model': False,
 '_compilation_job_name': None,
 '_is_edge_packaged_model': False,
 '_enable_network_isolation': False,
 'model_kms_key': None,
 'image_config': None}

In [9]:
%%time

model.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.m5.large'
)

Using already existing model: model-ACBOE


---------------!CPU times: user 251 ms, sys: 11.6 ms, total: 263 ms
Wall time: 7min 32s


In [10]:
model.endpoint_name

'model-ACBOE-2021-05-24-14-51-44-733'

In [11]:
from sagemaker import Predictor
predictor = Predictor(endpoint_name=model.endpoint_name)

In [12]:
boto_session = boto3.Session(region_name=region)
runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime', 
    region_name=region
)

In [13]:
feature_group_name = 'cookbook-feature-group'

record_response = runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString="5000"
)

In [14]:
record_response['Record']

[{'FeatureName': 'label', 'ValueAsString': '1'},
 {'FeatureName': 'a', 'ValueAsString': '-3.585282409963491'},
 {'FeatureName': 'b', 'ValueAsString': '-4.820820583530444'},
 {'FeatureName': 'c', 'ValueAsString': '88'},
 {'FeatureName': 'd', 'ValueAsString': '51'},
 {'FeatureName': 'index', 'ValueAsString': '5000'},
 {'FeatureName': 'event_time', 'ValueAsString': '1621862257.0'}]

In [15]:
test_record_list = [
    record_response['Record'][1]['ValueAsString'],
    record_response['Record'][2]['ValueAsString'],
    record_response['Record'][3]['ValueAsString'],
    record_response['Record'][4]['ValueAsString']
]

test_record_list

['-3.585282409963491', '-4.820820583530444', '88', '51']

In [16]:
csv_input = ','.join(test_record_list)
csv_input

'-3.585282409963491,-4.820820583530444,88,51'

In [17]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [18]:
predictor.predict(csv_input)

0.9837374687194824

In [20]:
endpoint_name = predictor.endpoint_name

%store endpoint_name
endpoint_name

Stored 'endpoint_name' (str)


'model-ACBOE-2021-05-24-14-51-44-733'

In [21]:
%store csv_input

Stored 'csv_input' (str)
